In [1]:
import pandas as pd
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report

from RandomForestClassifierGridSearch import RandomForestClassifierGridSearch
import pickle
pd.set_option('display.max_rows', None)

In [22]:
df = pd.read_parquet('../data/processed/scores_approvals_2020_2022.parquet')
df2 = pd.read_parquet('../data/processed/scores_approvals_2019_2021.parquet')

In [23]:
FEATURES = ['escore_bruto_p1_etapa1',
            'escore_bruto_p2_etapa1',
            'nota_redacao_etapa1',
            'escore_bruto_p1_etapa2',
            'escore_bruto_p2_etapa2',
            'nota_redacao_etapa2',
            'escore_bruto_p1_etapa3',
            'escore_bruto_p2_etapa3',
            'nota_redacao_etapa3',
            'cotista',
            'cotas_negros_flag',
            'publicas_flag',
            'publicas1_flag',
            'publicas2_flag', 'publicas3_flag', 'publicas4_flag', 'publicas5_flag',
            'publicas6_flag', 'publicas7_flag', 'publicas8_flag'
            ]

In [24]:
course_dummies = pd.get_dummies(df.course)
course_dummies_columns = list(course_dummies.columns)
df = pd.concat([df, course_dummies], axis=1)

In [25]:
FEATURES.extend(course_dummies_columns)

In [26]:
FEATURES

['escore_bruto_p1_etapa1',
 'escore_bruto_p2_etapa1',
 'nota_redacao_etapa1',
 'escore_bruto_p1_etapa2',
 'escore_bruto_p2_etapa2',
 'nota_redacao_etapa2',
 'escore_bruto_p1_etapa3',
 'escore_bruto_p2_etapa3',
 'nota_redacao_etapa3',
 'cotista',
 'cotas_negros_flag',
 'publicas_flag',
 'publicas1_flag',
 'publicas2_flag',
 'publicas3_flag',
 'publicas4_flag',
 'publicas5_flag',
 'publicas6_flag',
 'publicas7_flag',
 'publicas8_flag',
 'AGRONOMIA (BACHARELADO)',
 'ARQUITETURA E URBANISMO (BACHARELADO)',
 'ARQUIVOLOGIA (BACHARELADO)',
 'ARTES CÊNICAS - INTERPRETAÇÃO TEATRAL (BACHARELADO)',
 'ARTES VISUAIS (BACHARELADO)',
 'ARTES VISUAIS (LICENCIATURA)',
 'BIBLIOTECONOMIA (BACHARELADO)',
 'BIOTECNOLOGIA (BACHARELADO)',
 'CAMPUS UNB CEILÂNDIA (FCE) ENFERMAGEM (BACHARELADO)',
 'CAMPUS UNB PLANALTINA (FUP) – DIURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CIÊNCIA DA COMPUTAÇÃO (BACHARELADO)',
 'CIÊNCIA POLÍTICA (BACHARELA

In [6]:
len(FEATURES)

111

In [7]:
shuffled_df = df.sample(frac=1, random_state=42)
X = shuffled_df[FEATURES] # features
#X = df[FEATURES] # features
y = shuffled_df['label'] # labels
#y = df['label'] # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47) # 70% training and 30% test

In [8]:
print(X_train.shape)
print(y_train.shape)

(5810, 111)
(5810,)


In [9]:
y_train.value_counts()

0    4784
1    1026
Name: label, dtype: int64

In [10]:
course_dummies = pd.get_dummies(df2.course)
course_dummies_columns = list(course_dummies.columns)
df2['MÚSICA (BACHARELADO)'] = 0
df2 = pd.concat([df2, course_dummies], axis=1)
#df2 = df2[df2.label==1]
df2_shuffled = df2.sample(frac=1, random_state=42)

In [11]:
df2_shuffled.label.value_counts()

0    7276
1    1296
Name: label, dtype: int64

In [12]:
X_train = pd.concat([X_train, df2_shuffled[FEATURES]])
y_train = pd.concat([y_train, df2_shuffled['label']])

print(X_train.shape)
print(y_train.shape)

(14382, 111)
(14382,)


In [13]:
y_train.value_counts()

0    12060
1     2322
Name: label, dtype: int64

## Baseline Model

In [14]:
# Create a Random Forest Classifier
model = RandomForestClassifier(random_state=47)

# Fit randomized search
model = model.fit(X_train, y_train)

y_pred = model.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
classification_report(y_test, y_pred, output_dict=True)

Accuracy: 0.8616655196145905


{'0': {'precision': 0.8686186186186187,
  'recall': 0.978021978021978,
  'f1-score': 0.9200795228628231,
  'support': 1183},
 '1': {'precision': 0.7851239669421488,
  'recall': 0.35185185185185186,
  'f1-score': 0.4859335038363171,
  'support': 270},
 'accuracy': 0.8616655196145905,
 'macro avg': {'precision': 0.8268712927803837,
  'recall': 0.6649369149369149,
  'f1-score': 0.7030065133495701,
  'support': 1453},
 'weighted avg': {'precision': 0.8531034390228533,
  'recall': 0.8616655196145905,
  'f1-score': 0.8394054518806092,
  'support': 1453}}

In [12]:
 # Predict diabetes
predictions = model.predict_proba(X_test)
approval_prob = round(predictions[0][1], ndigits=3)

In [13]:
predictions[:, 1]

array([0.02, 0.38, 0.09, ..., 0.25, 0.04, 0.1 ])

In [14]:
predictions

array([[0.98, 0.02],
       [0.62, 0.38],
       [0.91, 0.09],
       ...,
       [0.75, 0.25],
       [0.96, 0.04],
       [0.9 , 0.1 ]])

## Hyperparameter tuning + class_weight

In [15]:
param_grid = {
    'n_estimators': [200, 300, 500],      # Number of trees in the forest
    'max_depth': [5, 10, 30],            # Maximum depth of each tree
    'min_samples_split': [5, 10, 30],       # Minimum number of samples required to split an internal node
    'min_samples_leaf': [8, 16, 32]          # Minimum number of samples required to be at a leaf node
}

In [16]:
# Assuming you have your training data X_train and corresponding labels y_train

# Instantiate the class
rf_gs = RandomForestClassifierGridSearch(X_train, y_train)

# Fit the classifier using GridSearchCV
rf_gs.fit(param_grid)

# Assuming you have your test data X_test and corresponding labels y_test

# Evaluate the best model on the test data
report = rf_gs.evaluate(X_test, y_test)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.79      0.86      1220
           1       0.40      0.75      0.52       233

    accuracy                           0.78      1453
   macro avg       0.67      0.77      0.69      1453
weighted avg       0.86      0.78      0.80      1453



In [ ]:
with open('model_flags_tuned.pickle','wb') as f:
    pickle.dump(model, f)

In [ ]:
#with open('model_flags_tuned.pickle','wb') as f:
#  pickle.dump(model, f)

## Stratified KFold + Hyperameter tuning + Flags

In [ ]:
#FEATURES.remove('argumento_final')
print(len(FEATURES))

X = df[FEATURES] # features
y = df['label'] # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47) # 70% training and 30% test

110


In [ ]:
# # Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],      # Number of trees in the forest
    'max_depth': [5, 10, 20, 40, 80],            # Maximum depth of each tree
    'min_samples_split': [5, 10, 20, 50],       # Minimum number of samples required to split an internal node
    'min_samples_leaf': [2, 8, 16, 32],          # Minimum number of samples required to be at a leaf node
}

In [ ]:
X_train.shape

(5810, 110)

In [ ]:
# Assuming you have your training data X_train and corresponding labels y_train

# Instantiate the class
rf_gs = RandomForestClassifierGridSearch(X_train, y_train)

# Fit the classifier using GridSearchCV
rf_gs.fit(param_grid)

# Assuming you have your test data X_test and corresponding labels y_test

# Evaluate the best model on the test data
report = rf_gs.evaluate(X_test, y_test)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1220
           1       0.57      0.64      0.60       233

    accuracy                           0.86      1453
   macro avg       0.75      0.77      0.76      1453
weighted avg       0.87      0.86      0.87      1453



In [ ]:
rf_gs.save_model('stratified_kfold_classweight15_tuned_model_spelling')

In [ ]:
with open('../ml_dev/models/stratified_kfold_classweight15_tuned_model_spelling.pickle', 'rb') as f:
  model_loaded = pickle.load(f)

c:\Anaconda\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Anaconda\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
y_pred = model_loaded.predict(X_test)
classification_report(y_test, y_pred, output_dict=True)

ValueError: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.


In [ ]:
def predict_approval(model, new_data):
    
    # Predict diabetes
    predictions = model.predict_proba(new_data)
    approval_prob = round(predictions[0][1], ndigits=3)

    #pred_to_label = {0: 'Negative', 1: 'Positive'}

    # Make a list of predictions
    #data = []
    #for t, pred in zip(new_data, predictions):
    #    data.append({'prediction': pred[0]})

    return approval_prob

In [ ]:
covariables = list({"escore_bruto_p1_etapa1": 5.172,
                            "escore_bruto_p2_etapa1": 14.653,
                            "nota_redacao_etapa1": 6.947,
                            "escore_bruto_p1_etapa2": 3.845,
                            "escore_bruto_p2_etapa2": 19.994,
                            "nota_redacao_etapa2": 7.222,
                            "escore_bruto_p1_etapa3": 4.998,
                            "escore_bruto_p2_etapa3": 16.66,
                            "nota_redacao_etapa3": 8.06,
                            "argumento_final": -25.699,
                            "2.1.1 CAMPUS  DARCY RIBEIRO – DIURNO  ADMINISTRAÇÃO (BACHARELADO)": 1.0,               
                            "cotista": 0.0,
                            "cotas_negros_flag": 0.0,
                            "publicas1_flag": 0.0,
                            "publicas2_flag": 0.0,
                            "publicas3_flag": 0.0,
                            "publicas4_flag": 0.0,
                            "publicas5_flag": 0.0,
                            "publicas6_flag": 0.0,
                            "publicas7_flag": 0.0,
                            "publicas8_flag": 0.0}.keys())

In [ ]:
from samples import sample_approved, sample_not_approved

In [ ]:
{'teste':sample_not_approved.get('cotista')}

{'teste': 0.0}

In [ ]:
list(sample_not_approved.keys())

['escore_bruto_p1_etapa1',
 'escore_bruto_p2_etapa1',
 'nota_redacao_etapa1',
 'escore_bruto_p1_etapa2',
 'escore_bruto_p2_etapa2',
 'nota_redacao_etapa2',
 'escore_bruto_p1_etapa3',
 'escore_bruto_p2_etapa3',
 'nota_redacao_etapa3',
 'argumento_final',
 '2.1.1 CAMPUS  DARCY RIBEIRO – DIURNO  ADMINISTRAÇÃO (BACHARELADO)',
 'cotista',
 'cotas_negros_flag',
 'publicas1_flag',
 'publicas2_flag',
 'publicas3_flag',
 'publicas4_flag',
 'publicas5_flag',
 'publicas6_flag',
 'publicas7_flag',
 'publicas8_flag']

In [ ]:
new_sample = sample_not_approved
#new_sample = pd.DataFrame([{col: new_sample.get(col) for col in FEATURES}])
#new_sample = new_sample.fillna(0)

In [ ]:
[True for key in new_sample.keys() if key in FEATURES else False]

SyntaxError: invalid syntax (Temp/ipykernel_17636/1785515755.py, line 1)

In [ ]:
[True for key in new_sample.keys()]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [ ]:
new_sample.to_dict()

{'escore_bruto_p1_etapa1': {0: 5.172},
 'escore_bruto_p2_etapa1': {0: 14.653},
 'nota_redacao_etapa1': {0: 6.947},
 'escore_bruto_p1_etapa2': {0: 3.845},
 'escore_bruto_p2_etapa2': {0: 19.994},
 'nota_redacao_etapa2': {0: 7.222},
 'escore_bruto_p1_etapa3': {0: 4.998},
 'escore_bruto_p2_etapa3': {0: 16.66},
 'nota_redacao_etapa3': {0: 8.06},
 'AGRONOMIA (BACHARELADO)': {0: 0},
 'ARQUITETURA E URBANISMO (BACHARELADO)': {0: 0},
 'ARQUIVOLOGIA (BACHARELADO)': {0: 0},
 'ARTES CÊNICAS - INTERPRETAÇÃO TEATRAL (BACHARELADO)': {0: 0},
 'ARTES VISUAIS (BACHARELADO)': {0: 0},
 'ARTES VISUAIS (LICENCIATURA)': {0: 0},
 'BIBLIOTECONOMIA (BACHARELADO)': {0: 0},
 'BIOTECNOLOGIA (BACHARELADO)': {0: 0},
 'CAMPUS UNB CEILÂNDIA (FCE) ENFERMAGEM (BACHARELADO)': {0: 0},
 'CAMPUS UNB PLANALTINA (FUP) – DIURNO CIÊNCIAS NATURAIS (LICENCIATURA)': {0: 0},
 'CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS NATURAIS (LICENCIATURA)': {0: 0},
 'CIÊNCIA DA COMPUTAÇÃO (BACHARELADO)': {0: 0},
 'CIÊNCIA POLÍTICA (BACHAREL

In [ ]:
FEATURES[33]

'DIURNO ADMINISTRAÇÃO (BACHARELADO)'

In [ ]:
sample_not_approved.get(FEATURES[33])

In [ ]:
predict_approval(model_loaded, new_data=new_sample)

0.104

In [ ]:
X_test.columns.to_list()

['escore_bruto_p1_etapa1',
 'escore_bruto_p2_etapa1',
 'nota_redacao_etapa1',
 'escore_bruto_p1_etapa2',
 'escore_bruto_p2_etapa2',
 'nota_redacao_etapa2',
 'escore_bruto_p1_etapa3',
 'escore_bruto_p2_etapa3',
 'nota_redacao_etapa3',
 'AGRONOMIA (BACHARELADO)',
 'ARQUITETURA E URBANISMO (BACHARELADO)',
 'ARQUIVOLOGIA (BACHARELADO)',
 'ARTES CÊNICAS - INTERPRETAÇÃO TEATRAL (BACHARELADO)',
 'ARTES VISUAIS (BACHARELADO)',
 'ARTES VISUAIS (LICENCIATURA)',
 'BIBLIOTECONOMIA (BACHARELADO)',
 'BIOTECNOLOGIA (BACHARELADO)',
 'CAMPUS UNB CEILÂNDIA (FCE) ENFERMAGEM (BACHARELADO)',
 'CAMPUS UNB PLANALTINA (FUP) – DIURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CIÊNCIA DA COMPUTAÇÃO (BACHARELADO)',
 'CIÊNCIA POLÍTICA (BACHARELADO)',
 'CIÊNCIAS AMBIENTAIS (BACHARELADO)',
 'CIÊNCIAS BIOLÓGICAS (BACHARELADO)',
 'CIÊNCIAS CONTÁBEIS (BACHARELADO)',
 'CIÊNCIAS ECONÔMICAS (BACHARELADO)',
 'CIÊNCIAS SOCIAIS – ANTROPOLOGIA / SOCIOLOGIA 

In [ ]:
my_sample = pd.DataFrame([X_test.iloc[0]])
#my_sample.rename_ax

In [ ]:
my_sample.rename(columns = {'AGRONOMIA (BACHARELADO)': 'AGRONOMIA 123 (BACHARELADO)'})

,escore_bruto_p1_etapa1,escore_bruto_p2_etapa1,nota_redacao_etapa1,escore_bruto_p1_etapa2,escore_bruto_p2_etapa2,nota_redacao_etapa2,escore_bruto_p1_etapa3,escore_bruto_p2_etapa3,nota_redacao_etapa3,AGRONOMIA 123 (BACHARELADO),...,cotista,cotas_negros_flag,publicas1_flag,publicas2_flag,publicas3_flag,publicas4_flag,publicas5_flag,publicas6_flag,publicas7_flag,publicas8_flag
510,1.149,21.549,5.808,3.845,25.857,8.273,1.428,25.228,8.148,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
my_sample.columns.to_list()

['escore_bruto_p1_etapa1',
 'escore_bruto_p2_etapa1',
 'nota_redacao_etapa1',
 'escore_bruto_p1_etapa2',
 'escore_bruto_p2_etapa2',
 'nota_redacao_etapa2',
 'escore_bruto_p1_etapa3',
 'escore_bruto_p2_etapa3',
 'nota_redacao_etapa3',
 'AGRONOMIA (BACHARELADO)',
 'ARQUITETURA E URBANISMO (BACHARELADO)',
 'ARQUIVOLOGIA (BACHARELADO)',
 'ARTES CÊNICAS - INTERPRETAÇÃO TEATRAL (BACHARELADO)',
 'ARTES VISUAIS (BACHARELADO)',
 'ARTES VISUAIS (LICENCIATURA)',
 'BIBLIOTECONOMIA (BACHARELADO)',
 'BIOTECNOLOGIA (BACHARELADO)',
 'CAMPUS UNB CEILÂNDIA (FCE) ENFERMAGEM (BACHARELADO)',
 'CAMPUS UNB PLANALTINA (FUP) – DIURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
 'CIÊNCIA DA COMPUTAÇÃO (BACHARELADO)',
 'CIÊNCIA POLÍTICA (BACHARELADO)',
 'CIÊNCIAS AMBIENTAIS (BACHARELADO)',
 'CIÊNCIAS BIOLÓGICAS (BACHARELADO)',
 'CIÊNCIAS CONTÁBEIS (BACHARELADO)',
 'CIÊNCIAS ECONÔMICAS (BACHARELADO)',
 'CIÊNCIAS SOCIAIS – ANTROPOLOGIA / SOCIOLOGIA 

In [ ]:
model_loaded.predict(new_sample)

array([0], dtype=int64)

## Stratified KFold + Hyperameter tuning + Simple Flags

In [ ]:
list(df.columns)

['numero_inscricao',
 'nome_x',
 'escore_bruto_p1_etapa1',
 'escore_bruto_p2_etapa1',
 'nota_redacao_etapa1',
 'escore_bruto_p1_etapa2',
 'escore_bruto_p2_etapa2',
 'nota_redacao_etapa2',
 'escore_bruto_p1_etapa3',
 'escore_bruto_p2_etapa3',
 'nota_redacao_etapa3',
 'argumento_final',
 'classificacao_final_universal',
 'classificacao_final_cotas_negros',
 'classificacao_final_publicas1',
 'classificacao_final_publicas2',
 'classificacao_final_publicas3',
 'classificacao_final_publicas4',
 'classificacao_final_publicas5',
 'classificacao_final_publicas6',
 'classificacao_final_publicas7',
 'classificacao_final_publicas8',
 'course',
 'cotista',
 'cotas_negros_flag',
 'publicas1_flag',
 'publicas2_flag',
 'publicas3_flag',
 'publicas4_flag',
 'publicas5_flag',
 'publicas6_flag',
 'publicas7_flag',
 'publicas8_flag',
 'nome_y',
 'campus',
 'curso',
 'periodo',
 '_merge',
 'label',
 'AGRONOMIA (BACHARELADO)',
 'ARQUITETURA E URBANISMO (BACHARELADO)',
 'ARQUIVOLOGIA (BACHARELADO)',
 'ARTES 

In [10]:
publicas_flags = [
 'publicas1_flag',
 'publicas2_flag',
 'publicas3_flag',
 'publicas4_flag',
 'publicas5_flag',
 'publicas6_flag',
 'publicas7_flag',
 'publicas8_flag']
df['publicas_flag'] = df[publicas_flags].sum(axis=1)

In [11]:
df.publicas_flag.value_counts()

0    5506
2     771
1     672
4     314
Name: publicas_flag, dtype: int64

In [12]:
FEATURES = [col for col in FEATURES if col not in publicas_flags] 
print(len(FEATURES))

102


In [13]:
FEATURES.append('publicas_flag')

In [14]:
print(len(FEATURES))


103


In [16]:
X = df[FEATURES] # features
y = df['label'] # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47) # 70% training and 30% test

In [ ]:
# Assuming you have your training data X_train and corresponding labels y_train

# Instantiate the class
rf_gs = RandomForestClassifierGridSearch(X_train, y_train)

# Fit the classifier using GridSearchCV
rf_gs.fit(param_grid)

# Assuming you have your test data X_test and corresponding labels y_test

# Evaluate the best model on the test data
report = rf_gs.evaluate(X_test, y_test)
print(report)

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      1220
           1       0.58      0.58      0.58       233

    accuracy                           0.87      1453
   macro avg       0.75      0.75      0.75      1453
weighted avg       0.87      0.87      0.87      1453



## XGBoost

In [15]:
import xgboost as xgb

# Calculate the class imbalance ratio
class_ratio = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
class_ratio

5.1937984496124034

In [16]:
# Define the XGBoost parameters
params = {
    'booster': 'gbtree',
    'n_estimators': 100,
    'eta': 0.1,                        # Learning rate
    'max_depth': 16,                   # Maximum depth of a tree
    'subsample': 0.8,                  # Subsample ratio of the training instances
    #'colsample_bytree': 0.8,          # Subsample ratio of columns when constructing each tree
    'scale_pos_weight': class_ratio,
    #'gamma': 0.5
}

In [17]:
# Create an XGBoost classifier
model = xgb.XGBClassifier(**params)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
classification_report(y_test, y_pred, output_dict=True)

Accuracy: 0.8816242257398486


{'0': {'precision': 0.946949602122016,
  'recall': 0.9053254437869822,
  'f1-score': 0.9256698357821953,
  'support': 1183},
 '1': {'precision': 0.6521739130434783,
  'recall': 0.7777777777777778,
  'f1-score': 0.7094594594594594,
  'support': 270},
 'accuracy': 0.8816242257398486,
 'macro avg': {'precision': 0.7995617575827472,
  'recall': 0.8415516107823799,
  'f1-score': 0.8175646476208274,
  'support': 1453},
 'weighted avg': {'precision': 0.8921736654040496,
  'recall': 0.8816242257398486,
  'f1-score': 0.8854930968922168,
  'support': 1453}}

In [18]:
with open('xgboost_2019_2022.pickle','wb') as f:
    pickle.dump(model, f)

In [2]:
with open('../ml_dev/models/xgboost_2019_2022.pickle', 'rb') as f:
  model_loaded = pickle.load(f)

In [19]:
# Create an XGBoost classifier
model = xgb.XGBClassifier(**params)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
classification_report(y_test, y_pred, output_dict=True)

Accuracy: 0.8781830695113558


{'0': {'precision': 0.935672514619883,
  'recall': 0.9180327868852459,
  'f1-score': 0.9267687215556475,
  'support': 1220},
 '1': {'precision': 0.609375,
  'recall': 0.6695278969957081,
  'f1-score': 0.6380368098159509,
  'support': 233},
 'accuracy': 0.8781830695113558,
 'macro avg': {'precision': 0.7725237573099415,
  'recall': 0.793780341940477,
  'f1-score': 0.7824027656857993,
  'support': 1453},
 'weighted avg': {'precision': 0.883348136845325,
  'recall': 0.8781830695113558,
  'f1-score': 0.8804682842291854,
  'support': 1453}}

In [37]:
user_features = ['escore_bruto_p1_etapa1', 'escore_bruto_p2_etapa1',
       'nota_redacao_etapa1', 'escore_bruto_p1_etapa2',
       'escore_bruto_p2_etapa2', 'nota_redacao_etapa2',
       'escore_bruto_p1_etapa3', 'escore_bruto_p2_etapa3',
       'nota_redacao_etapa3', 'cotista', 'cotas_negros_flag', 'publicas_flag',
       'publicas1_flag', 'publicas2_flag', 'publicas3_flag', 'publicas4_flag',
       'publicas5_flag', 'publicas6_flag', 'publicas7_flag', 'publicas8_flag']

In [50]:
#X_test[user_features].iloc[0,:].to_dict()
#X_test.iloc[0,:].to_dict()

In [3]:
from samples import sample_approved, sample_not_approved

In [5]:
FEATURES = ['escore_bruto_p1_etapa1',
            'escore_bruto_p2_etapa1',
            'nota_redacao_etapa1',
            'escore_bruto_p1_etapa2',
            'escore_bruto_p2_etapa2',
            'nota_redacao_etapa2',
            'escore_bruto_p1_etapa3',
            'escore_bruto_p2_etapa3',
            'nota_redacao_etapa3',
            'cotista',
            'cotas_negros_flag',
            'publicas_flag',
            'publicas1_flag',
            'publicas2_flag',
            'publicas3_flag',
            'publicas4_flag',
            'publicas5_flag',
            'publicas6_flag',
            'publicas7_flag',
            'publicas8_flag',
            'AGRONOMIA (BACHARELADO)',
            'ARQUITETURA E URBANISMO (BACHARELADO)',
            'ARQUIVOLOGIA (BACHARELADO)',
            'ARTES CÊNICAS - INTERPRETAÇÃO TEATRAL (BACHARELADO)',
            'ARTES VISUAIS (BACHARELADO)',
            'ARTES VISUAIS (LICENCIATURA)',
            'BIBLIOTECONOMIA (BACHARELADO)',
            'BIOTECNOLOGIA (BACHARELADO)',
            'CAMPUS UNB CEILÂNDIA (FCE) ENFERMAGEM (BACHARELADO)',
            'CAMPUS UNB PLANALTINA (FUP) – DIURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
            'CAMPUS UNB PLANALTINA (FUP) – NOTURNO CIÊNCIAS NATURAIS (LICENCIATURA)',
            'CIÊNCIA DA COMPUTAÇÃO (BACHARELADO)',
            'CIÊNCIA POLÍTICA (BACHARELADO)',
            'CIÊNCIAS AMBIENTAIS (BACHARELADO)',
            'CIÊNCIAS BIOLÓGICAS (BACHARELADO)',
            'CIÊNCIAS CONTÁBEIS (BACHARELADO)',
            'CIÊNCIAS ECONÔMICAS (BACHARELADO)',
            'CIÊNCIAS SOCIAIS – ANTROPOLOGIA / SOCIOLOGIA (BACHARELADO/LICENCIATURA)',
            'COMPUTAÇÃO (LICENCIATURA)',
            'COMUNICAÇÃO ORGANIZACIONAL (BACHARELADO)',
            'COMUNICAÇÃO SOCIAL - PUBLICIDADE E PROPAGANDA (BACHARELADO)',
            'COMUNICAÇÃO SOCIAL – AUDIOVISUAL (BACHARELADO)',
            'DESIGN – PROGRAMAÇÃO VISUAL/PROJETO DO PRODUTO (BACHARELADO)',
            'DIREITO (BACHARELADO)',
            'DIURNO ADMINISTRAÇÃO (BACHARELADO)',
            'EDUCAÇÃO FÍSICA (BACHARELADO)',
            'EDUCAÇÃO FÍSICA (LICENCIATURA)',
            'ENFERMAGEM (BACHARELADO)',
            'ENGENHARIA AMBIENTAL (BACHARELADO)',
            'ENGENHARIA CIVIL (BACHARELADO)',
            'ENGENHARIA DE COMPUTAÇÃO (BACHARELADO)',
            'ENGENHARIA DE PRODUÇÃO (BACHARELADO)',
            'ENGENHARIA DE REDES DE COMUNICAÇÃO (BACHARELADO)',
            'ENGENHARIA ELÉTRICA (BACHARELADO)',
            'ENGENHARIA FLORESTAL (BACHARELADO)',
            'ENGENHARIA MECATRÔNICA – CONTROLE E AUTOMAÇÃO (BACHARELADO)',
            'ENGENHARIA MECÂNICA (BACHARELADO)',
            'ENGENHARIA QUÍMICA (BACHARELADO)',
            'ENGENHARIAS – AEROESPACIAL / AUTOMOTIVA / ELETRÔNICA / ENERGIA / SOFTWARE (BACHARELADOS)',
            'ESTATÍSTICA (BACHARELADO)',
            'FARMÁCIA (BACHARELADO)',
            'FILOSOFIA (BACHARELADO/LICENCIATURA)',
            'FILOSOFIA (LICENCIATURA)',
            'FISIOTERAPIA (BACHARELADO)',
            'FONOAUDIOLOGIA (BACHARELADO)',
            'FÍSICA (BACHARELADO)',
            'GEOFÍSICA (BACHARELADO)',
            'GEOGRAFIA (BACHARELADO/LICENCIATURA)',
            'GEOLOGIA (BACHARELADO)',
            'GESTÃO AMBIENTAL (BACHARELADO)',
            'GESTÃO DE AGRONEGÓCIO (BACHARELADO)',
            'GESTÃO DE POLÍTICAS PÚBLICAS (BACHARELADO)',
            'GESTÃO DO AGRONEGÓCIO (BACHARELADO)',
            'HISTÓRIA (BACHARELADO/LICENCIATURA )',
            'HISTÓRIA (LICENCIATURA)',
            'JORNALISMO (BACHARELADO)',
            'LETRAS – PORTUGUÊS DO BRASIL COMO SEGUNDA LÍNGUA (LICENCIATURA)',
            'LETRAS – TRADUÇÃO ESPANHOL (BACHARELADO)',
            'LETRAS – TRADUÇÃO – FRANCÊS (BACHARELADO)',
            'LETRAS – TRADUÇÃO – INGLÊS (BACHARELADO)',
            'LICENCIATURA EM ARTES CÊNICAS',
            'LICENCIATURA EM CIÊNCIAS BIOLÓGICAS',
            'LICENCIATURA EM FÍSICA',
            'LICENCIATURA EM MATEMÁTICA',
            'LICENCIATURA EM QUÍMICA',
            'LÍNGUA E LITERATURA JAPONESA (LICENCIATURA)',
            'LÍNGUA ESPANHOLA E LITERATURA ESPANHOLA E HISPANO - AMERICANA (LICENCIATURA)',
            'LÍNGUA ESTRANGEIRA APLICADA – MULTILINGUISMO E SOCIEDADE DA INFORMAÇÃO (BACHARELADO)',
            'LÍNGUA FRANCESA E RESPECTIVA LITERATURA (BACHARELADO/LICENCIATURA)',
            'LÍNGUA INGLESA E RESPECTIVA LITERATURA (BACHARELADO/LICENCIATURA)',
            'LÍNGUA PORTUGUESA E RESPECTIVA LITERATURA (BACHARELADO/LICENCIATURA)',
            'LÍNGUA PORTUGUESA E RESPECTIVA LITERATURA (LICENCIATURA)',
            'MATEMÁTICA (BACHARELADO/LICENCIATURA)',
            'MEDICINA (BACHARELADO)',
            'MEDICINA VETERINÁRIA (BACHARELADO)',
            'MUSEOLOGIA (BACHARELADO)',
            'MÚSICA (BACHARELADO)',
            'MÚSICA (LICENCIATURA)',
            'NOTURNO ADMINISTRAÇÃO (BACHARELADO)',
            'NUTRIÇÃO (BACHARELADO)',
            'ODONTOLOGIA (BACHARELADO)',
            'PEDAGOGIA (LICENCIATURA)',
            'PSICOLOGIA (BACHARELADO / LICENCIATURA / PSICÓLOGO)',
            'QUÍMICA (BACHARELADO)',
            'QUÍMICA TECNOLÓGICA (BACHARELADO)',
            'RELAÇÕES INTERNACIONAIS (BACHARELADO)',
            'SAÚDE COLETIVA (BACHARELADO)',
            'SERVIÇO SOCIAL (BACHARELADO)',
            'TEORIA, CRÍTICA E HISTÓRIA DA ARTE (BACHARELADO)',
            'TERAPIA OCUPACIONAL (BACHARELADO)',
            'TURISMO (BACHARELADO)']

In [8]:
def preprocess_input_features(features: dict) -> pd.DataFrame:
    new_sample = features
    new_sample = pd.DataFrame([{col: new_sample.get(col) for col in FEATURES}])
    new_sample = new_sample.fillna(0)
    return new_sample

new_data = sample_not_approved
new_data = preprocess_input_features(new_data)

# Predict diabetes
#predictions = model.predict_proba(new_data)
model_loaded.predict_proba(new_data)
#approval_prob = predictions[0][1]

array([[0.5663873 , 0.43361273]], dtype=float32)

In [21]:
# Create an XGBoost classifier
model = xgb.XGBClassifier(scale_pos_weight=class_ratio)

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [4, 8, 16, 32],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=4, scoring='precision')
grid_search.fit(X_train, y_train)

# Make predictions on the test set using the best model
y_pred = grid_search.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
classification_report(y_test, y_pred, output_dict=True)

Accuracy: 0.8802477632484514


{'0': {'precision': 0.9231391585760518,
  'recall': 0.9352459016393443,
  'f1-score': 0.9291530944625408,
  'support': 1220},
 '1': {'precision': 0.6359447004608295,
  'recall': 0.592274678111588,
  'f1-score': 0.6133333333333333,
  'support': 233},
 'accuracy': 0.8802477632484514,
 'macro avg': {'precision': 0.7795419295184407,
  'recall': 0.7637602898754661,
  'f1-score': 0.771243213897937,
  'support': 1453},
 'weighted avg': {'precision': 0.8770852640537897,
  'recall': 0.8802477632484514,
  'f1-score': 0.8785089070275063,
  'support': 1453}}